In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

ImportError: cannot import name 'display' from 'IPython.core.display' (C:\Users\santi\AppData\Roaming\Python\Python311\site-packages\IPython\core\display.py)

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import re
from sklearn.model_selection import train_test_split

import nltk
from nltk.stem import WordNetLemmatizer

nltk.download("wordnet")
nltk.download("omw-1.4")

lemmatizer = WordNetLemmatizer()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
# Your code



# check column names
print(data.head())

# split between text and label
X = data["text"] 
y = data["label"]


X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,       
    random_state=42,
    # to include similar proportions of spam and mail
    stratify=y           
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

print("\ndistribution in train:")
print(y_train.value_counts(normalize=True))

print("\ndistribution in test:")
print(y_test.value_counts(normalize=True))



                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0
Train size: (800,)
Test size: (200,)

distribution in train:
label
0    0.5575
1    0.4425
Name: proportion, dtype: float64

distribution in test:
label
0    0.56
1    0.44
Name: proportion, dtype: float64


## Data Preprocessing

In [5]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

import re

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
# Your code

def clean_text_1(text):
    # JS
    text = re.sub(r"<script.*?>.*?</script>", " ", text, flags=re.DOTALL | re.IGNORECASE)
    # CSS
    text = re.sub(r"<style.*?>.*?</style>", " ", text, flags=re.DOTALL | re.IGNORECASE)
    # html comments
    text = re.sub(r"<!--.*?-->", " ", text, flags=re.DOTALL)
    # rest of tags
    text = re.sub(r"<[^>]+>", " ", text)
    
    return text
   
X_train_clean = X_train.apply(clean_text_1)
X_test_clean = X_test.apply(clean_text_1)

# print("ORIGINAL:\n", X_train.iloc[1])
# print("\nCLEANED:\n", X_train_clean.iloc[1])


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
def clean_text_2(text):
    # Remove all special characters, numbers and punctuation (keep letters only)
    text = re.sub(r"[^a-zA-Z]", " ", text)

    # Remove all single characters (surrounded by spaces)
    text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)

    # Remove single characters from the start
    text = re.sub(r"^\s*[a-zA-Z]\s+", " ", text)

    # Substitute multiple spaces with single space
    text = re.sub(r"\s+", " ", text)

    # Remove prefixed 'b'
    text = re.sub(r"^b\s+", "", text)

    # Convert to lowercase
    return text.lower().strip()


X_train_clean = X_train_clean.apply(clean_text_2)
X_test_clean = X_test_clean.apply(clean_text_2)

## Now let's work on removing stopwords
Remove the stopwords.

In [8]:
# Your code

stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    email_words = text.split()
    email_words = [word for word in email_words if word not in stop_words]
    return " ".join(email_words)

X_train_clean = X_train_clean.apply(remove_stopwords)
X_test_clean = X_test_clean.apply(remove_stopwords)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [9]:
# Your code
def lemmatize_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

X_train_clean = X_train_clean.apply(lemmatize_text)
X_test_clean  = X_test_clean.apply(lemmatize_text)

i = 1
print("FINAL CLEANED:\n", X_train_clean.iloc[i])

FINAL CLEANED:
 mr henry kaborethe chief auditor inchargeforeign remittance unit african development bank adb ouagadougou burkina faso dear partner presumed well family please let surprise message got contact information fromthe international directory week ago decided contact magintude lucrative transaction present future survival life moreover laid solemn trust decided disclose successful confidential transaction chief auditor incharge foreign remittance unit bank decided contact financial transaction worth sum nineteen million three hundred thousand united state america present future success abandoned fund belongs one bank foreign customer died along entire family plane crash disaster since year ago meanwhile fortune came across deceased file arranging old abandoned customer file sign submit entire bank management official documentation audit year informed clearly stated foreign banking rule regulation signed lawfully fund remains unclaimed till period year started date beneficiary

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [10]:
# Your code

X_train_ham  = X_train_clean[y_train == 0]
X_train_spam = X_train_clean[y_train == 1]

# top_10_ham
vectorizer_ham = CountVectorizer()
X_ham_bow = vectorizer_ham.fit_transform(X_train_ham)

ham_word_counts = X_ham_bow.sum(axis=0)
ham_words_freq = zip(
    vectorizer_ham.get_feature_names_out(),
    ham_word_counts.A1
)

ham_words_freq = sorted(ham_words_freq, key=lambda x: x[1], reverse=True)
top_10_ham = ham_words_freq[:10]

print(top_10_ham)

# top_10_spam
vectorizer_spam = CountVectorizer()
X_spam_bow = vectorizer_spam.fit_transform(X_train_spam)

spam_word_counts = X_spam_bow.sum(axis=0)
spam_words_freq = zip(
    vectorizer_spam.get_feature_names_out(),
    spam_word_counts.A1
)

spam_words_freq = sorted(spam_words_freq, key=lambda x: x[1], reverse=True)
top_10_spam = spam_words_freq[:10]

print(top_10_spam)


[('state', 97), ('president', 95), ('would', 92), ('mr', 90), ('obama', 80), ('percent', 80), ('call', 77), ('work', 72), ('time', 70), ('one', 69)]
[('money', 761), ('account', 674), ('bank', 615), ('fund', 600), ('transaction', 435), ('business', 412), ('country', 401), ('nbsp', 387), ('mr', 384), ('million', 364)]


## Extra features

In [11]:
""" # We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()"""

money_symbol_list = "|".join(["euro", "dollar", "pound", "€", r"\$"])

suspicious_words = "|".join([
    "free", "cheap", "sex", "money", "account", "bank",
    "fund", "transfer", "transaction", "win", "deposit", "password"
])

X_train_money_mark = X_train_clean.str.contains(money_symbol_list, regex=True).astype(int)
X_train_suspicious_words = X_train_clean.str.contains(suspicious_words, regex=True).astype(int)
X_train_text_len = X_train_clean.apply(len)

X_test_money_mark = X_test_clean.str.contains(money_symbol_list, regex=True).astype(int)
X_test_suspicious_words = X_test_clean.str.contains(suspicious_words, regex=True).astype(int)
X_test_text_len = X_test_clean.apply(len)



## How would work the Bag of Words with Count Vectorizer concept?

In [12]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# train
X_train_bow = vectorizer.fit_transform(X_train_clean)
# same as 
# vectorizer.fit(X_train_clean)
# X_train_bow = vectorizer.transform(X_train_clean)

# test
X_test_bow = vectorizer.transform(X_test_clean)

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [13]:
# Your code
# load vectorizer
tfidf_vectorizer = TfidfVectorizer()

#vectorize dataset
#train
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_clean)
#test
X_test_tfidf  = tfidf_vectorizer.transform(X_test_clean)


## And the Train a Classifier?

In [14]:
# Your code
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

#init classifier
clf = LogisticRegression(max_iter=1000)

#train model 
clf.fit(X_train_tfidf, y_train)

# predict for test
y_pred = clf.predict(X_test_tfidf)



print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.985
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       112
           1       1.00      0.97      0.98        88

    accuracy                           0.98       200
   macro avg       0.99      0.98      0.98       200
weighted avg       0.99      0.98      0.98       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [15]:
# Your code